In [17]:
import pandas as pd
df=pd.read_csv(r"../Data/cleaned_data.csv")
df.head()



,product_href,product_name,product_type,brand,price,description,picture_src,Sensitive,Combination,Oily,Dry,Normal,skintype_list,notable_effects_list,product_id
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,209000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,0,0,1,0,0,['Oily'],"['Acne-Free', 'Pore-Care', 'Brightening', 'Ant...",prod_0
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,181800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,0,1,0,1,1,"['Normal', 'Dry', 'Combination']","['Soothing', 'Balancing']",prod_1
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,149000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",1,1,1,1,1,"['Normal', 'Dry', 'Oily', 'Combination', 'Sens...","['Soothing', 'Balancing']",prod_2
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,290000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,0,0,1,0,0,['Oily'],"['Acne-Free', 'Pore-Care', 'Brightening', 'Ant...",prod_3
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,194650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",0,0,0,1,1,"['Normal', 'Dry']","['Brightening', 'Soothing']",prod_4


In [18]:
def combine_text(row):
    return (
        f"Product: {row['product_name']}. "
        f"Type: {row['product_type']}. "
        f"Brand: {row['brand']}. "
        f"Description: {row['description']}. "
        f"Skin Types: {', '.join(row['skintype_list'])}. "
        f"Effects: {', '.join(row['notable_effects_list'])}."
    )

df['combined_text'] = df.apply(combine_text, axis=1)


In [19]:
df['combined_text']

0       Product: ACWELL Bubble Free PH Balancing Clean...
1       Product: ACWELL pH Balancing Soothing Cleansin...
2       Product: Acwell Licorice pH Balancing Cleansin...
3       Product: ACWELL Aquaseal Soothing Tonic . Type...
4       Product: Licorice pH Balancing Essence Mist. T...
                              ...                        
1188    Product: Whitelab Brightening Day Cream . Type...
1189    Product: YOU The Radiance White Purifying Faci...
1190    Product: YOU Basic Skin Care Simply Fresh and ...
1191    Product: YOU Dazzling Glow Up Facial Foam. Typ...
1192    Product: YOU Golden Age Deep Cleansing Facial ...
Name: combined_text, Length: 1193, dtype: object

In [20]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


In [21]:
#!pip install sentence-transformers



In [22]:
 # convert the column to a list of strings
corpus = df['combined_text'].tolist() 


In [23]:
embeddings = model.encode(corpus, show_progress_bar=True)


Batches: 100%|██████████| 38/38 [00:52<00:00,  1.39s/it]


In [24]:
df['embedding'] = embeddings.tolist()


In [25]:
import numpy as np
np.save('../Embeddings/product_embeddings.npy', embeddings)
df.to_pickle('../Embeddings/product_data_with_embeddings.pkl')


In [26]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def semantic_search(query, model, df, embeddings, top_k=5):
    # Encode the user query into a vector
    query_vec = model.encode([query])
    
    # Calculate cosine similarity between the query vector and product embeddings
    similarities = cosine_similarity(query_vec, embeddings)[0]
    
    # Get the indices of top-k most similar products
    top_indices = similarities.argsort()[-top_k:][::-1]
    
    # Return the top-k products with similarity scores
    results = df.iloc[top_indices].copy()
    results['similarity'] = similarities[top_indices]
    return results[['product_name', 'brand', 'product_type', 'description', 'similarity']]


In [27]:
query="hydrating moisturizer for dry skin"
output=semantic_search(query,model,df,embeddings,top_k=5)
print(output)

                                          product_name       brand  \
213  CLINIQUE Moisture Surge 100Hour Auto-Replenish...   CLINIQUE    
841  SAFI Dermasafe Soothe & Hydrate Day Moisturize...       SAFI    
337   Erha ERHA X AQUA Re-fresh Hydrating Face Mist...        ERHA   
292        ELSHE SKIN Hydra Boost Oil Free Moisturizer  ELSHE SKIN   
291      ELSHE SKIN Deep Hydrating Calming Moisturizer  ELSHE SKIN   

    product_type                                        description  \
213  Moisturizer  What it is: An oil-free gel-cream moisturiser ...   
841  Moisturizer  Krim pagi dengan tekstur gel yang mengandung A...   
337  Moisturizer  Hydrating Face Mist – Penyegar wajah dengan ca...   
292  Moisturizer                                             #NAME?   
291  Moisturizer  -Hydrate, Soothing and Moisturize -Mencegah TE...   

     similarity  
213    0.594770  
841    0.560025  
337    0.541834  
292    0.536644  
291    0.535172  
